<style>
    /* Main container style */
    .note-box {
        background-color: #1e1e2e;       /* Dark Blue-Grey Background */
        color: #cdd6f4;                  /* Soft White Text */
        border-left: 6px solid #89b4fa;  /* Blue Accent Border */
        border-radius: 8px;
        padding: 20px;
        margin: 20px 0;
        font-family: system-ui, -apple-system, sans-serif;
        line-height: 1.6;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2);
        box-sizing: border-box;
        max-width: 100%;
        overflow-wrap: break-word;
    }
    
    /* Header style */
    .note-box h2 {
        color: #89b4fa;                  /* Blue Header */
        margin-top: 0;
        margin-bottom: 15px;
        font-size: 1.6rem;
        font-weight: 600;
        border-bottom: 1px solid #45475a;
        padding-bottom: 10px;
    }

    /* Important keywords */
    .note-box strong {
        color: #f9e2af;                  /* Soft Gold/Yellow */
        font-weight: 600;
    }

    /* Inline code snippets */
    .note-box .code-inline {
        background-color: #313244;
        color: #f38ba8;                  /* Soft Red/Pink */
        padding: 2px 6px;
        border-radius: 4px;
        font-family: 'Menlo', 'Consolas', monospace;
        font-size: 0.9em;
        border: 1px solid #45475a;
        white-space: pre-wrap;
    }

    /* Lists */
    .note-box ul {
        padding-left: 20px;
        margin: 10px 0;
    }
    .note-box li {
        margin-bottom: 8px;
    }
</style>
<div class="note-box">
    <h2>Chapter4.1: Fundamentals of Recurrent Neural Netwoks(RNN)</h2>
    <p>
    <strong>Objective</strong>:undrestand how RNNs process sequential data by maintaining a "memory" (hidden state).
    we will implement a vanilla RNN from scratch in Pytorch to predict a time-series sine wave.</p>
    <p><strong>Key Concept</strong>: Sequence Data, Hiden State, Backpropagation Through Time(BTT).</p>

In [5]:
import torch
import torch.nn as nn
import numpy as np 
import matplotlib.pyplot as plt

SEQ_LENGTH = 20 # how many pass points the model sees
NUM_SAMPLES = 1000 #total data points
TEST_SIZE = 0.2 #20% for testing

#1.Generate Synthetic Data(sine waves)
# we add a little noise to make it more realistic 
t = np.linspace(0, 100, NUM_SAMPLES)
data = np.sin(t)
np.random.randn(NUM_SAMPLES) 

# 2. Prepare Dat Windows
def create_sequences(data, seq_lenght):
    xs, ys =[], []
    for i in range(len(data) - seq_lenght):
        x = data[i:i+seq_lenght]
        y = data[i+seq_lenght]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)


X, y = create_sequences(data, SEQ_LENGTH)

# 3.Convert to Pytorch Tensor 
# RNN expects the following input shape( Batch_Size, Seq_lenght, Input_size)
train_size = int(len(X)*(1-TEST_SIZE))

X_train = torch.from_numpy(X[:train_size]).float().unsqueeze(-1)#add input size dim
y_train = torch.from_numpy(y[:train_size]).float().unsqueeze(-1)#add input size dim
X_test = torch.from_numpy(X[train_size:]).float().unsqueeze(-1)#add input size dim
y_test = torch.from_numpy(y[train_size:]).float().unsqueeze(-1)#add input size dim
print(f"Input Shape:{X_train.shape}->(Batch, Sequence, Features)")


Input Shape:torch.Size([784, 20, 1])->(Batch, Sequence, Features)


<style>
    /* Main container style */
    .note-box {
        background-color: #1e1e2e;       /* Dark Blue-Grey Background */
        color: #cdd6f4;                  /* Soft White Text */
        border-left: 6px solid #89b4fa;  /* Blue Accent Border */
        border-radius: 8px;
        padding: 20px;
        margin: 20px 0;
        font-family: system-ui, -apple-system, sans-serif;
        line-height: 1.6;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2);
        box-sizing: border-box;
        max-width: 100%;
        overflow-wrap: break-word;
    }
    
    /* Header style */
    .note-box h2 {
        color: #89b4fa;                  /* Blue Header */
        margin-top: 0;
        margin-bottom: 15px;
        font-size: 1.6rem;
        font-weight: 600;
        border-bottom: 1px solid #45475a;
        padding-bottom: 10px;
    }

    /* Important keywords */
    .note-box strong {
        color: #f9e2af;                  /* Soft Gold/Yellow */
        font-weight: 600;
    }

    /* Inline code snippets */
    .note-box .code-inline {
        background-color: #313244;
        color: #f38ba8;                  /* Soft Red/Pink */
        padding: 2px 6px;
        border-radius: 4px;
        font-family: 'Menlo', 'Consolas', monospace;
        font-size: 0.9em;
        border: 1px solid #45475a;
        white-space: pre-wrap;
    }

    /* Lists */
    .note-box ul {
        padding-left: 20px;
        margin: 10px 0;
    }
    .note-box li {
        margin-bottom: 8px;
    }
</style>
<div class ="note-box">
    <h2> The Math: Inside the Hidden State</h2>
    <p> Unlike standard Feed-Forward networks, an RNN processes data sequentially. At every time step t, the netwok tales two inputs:
        <ul>
            <li><strong>The current input data:</strong>x_t</li>
            <li><strong>The previous hidden state:</strong>h{t-1}(Context from the past)</li>
            <li><strong>The new hidden state:</strong>h_t is calculated as follow:</li>
    </p>


$$ 

h_t = \tanh(W-{ih} x_t +b_{ih}+ W_{hh} H_{t-1}+ b_{hh})

$$
<p> the output y_t is then derived from this hidden state:</p>

$$

y_t=W_{hy} h_t + b_{y}

$$

</div>

In [8]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        #the RNN layer
        #batch_first = True means that the input format is (batch, seq, features)
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)

        #the linear layer(decoder)
        #maps the final hidden state to the output value 
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        #initialize the hidden state with zeros 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:,-1,:]
        out=self.fc(out)
        return out
    
input_size =1 
hidden_size = 32
output_size = 1
num_layers = 1
model = SimpleRNN(input_size, hidden_size, output_size, num_layers)
print(model)


SimpleRNN(
  (rnn): RNN(1, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)
